<a href="https://colab.research.google.com/github/JasonLimJS/Grab-AI-for-SEA/blob/master/Model%201%20of%20Ensemble%20kNN%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from scipy import stats
import time
import pickle

from sklearn.externals import joblib

drive.mount('drive')

Define function to train and test kNN model :

In [0]:
def train_test_kNN(X_train,X_test,y_train,y_test, Model, max_neighbor):

  kNN= KNeighborsRegressor()

  params_kNN= {'n_neighbors':list(range(1,max_neighbor+1)), 'weights':['uniform','distance']}

  grid_kNN = GridSearchCV(estimator=kNN,
                   param_grid= params_kNN,
                   scoring='neg_mean_squared_error',
                   cv=5,
                   n_jobs=-1)

  start_time= time.time()
  grid_kNN.fit(X_train,y_train)
  end_time= time.time() - start_time

  best_kNN= grid_kNN.best_estimator_
  y_pred= best_kNN.predict(X_test)
  r2= best_kNN.score(X_test,y_test)
  rmse= MSE(y_test,y_pred)**0.5


  print(Model + ' uses ' + str(end_time) + ' secs to train, having R2 value of ' + str(r2) + ' and RSME of ' + str(rmse))
  print('The best model is of this type is: ' + str(best_kNN))

Model 1

In [0]:
raw= pd.read_csv('drive/My Drive/raw.csv')

raw.sort_values(by=['geohash6','day','time_min'],inplace=True)
raw.index= raw.geohash6.values
last_day= pd.DataFrame({'last_day':raw.groupby('geohash6').day.tail(1)})
start_day= last_day - 14
start_day.columns= ['start_day']
raw_last= pd.merge(left=raw,right=last_day,left_index=True,right_index=True)
raw_start_last= pd.merge(left=raw_last,right=start_day,left_index=True,right_index=True)

last_15d= raw_start_last[np.logical_and(raw_start_last.day>=raw_start_last.start_day,raw_start_last.day<=raw_start_last.last_day)]

last_15d.to_csv('last_15d.csv')
!cp last_15d.csv drive/My\ Drive/

last_15d= pd.read_csv('drive/My Drive/last_15d.csv')

arranged_col= ['lat','long','time_min']
X= last_15d[arranged_col].values
y= last_15d.demand.values
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state= 7, \
                                                   test_size=0.35,stratify=X[:,2])
norm= StandardScaler()

X_train_scaled= norm.fit_transform(X_train)
norm_mean= norm.mean_
norm_std= norm.var_**0.5
X_test_scaled= norm.transform(X_test)

Model1= "Model 1"

train_test_kNN(X_train_scaled,X_test_scaled,y_train,y_test, Model1, 60)

######################## Best Model after Hyperparameter Tuning is : #######################

#Training time: 3713.8608691692352 secs, R2 value: 0.8745999622040456, RSME: 0.058954128844778846

#KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
#metric_params=None, n_jobs=None, n_neighbors=22, p=2,
#weights='uniform')

#norm_mean= np.array([ -5.34803514,  90.76456439, 611.01511076])
#norm_std= np.array([5.74593198e-02, 1.02564882e-01, 3.91570353e+02])


best_model1= KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                   metric_params=None, n_jobs=None, n_neighbors=22, p=2,
                    weights='uniform')

best_model1.fit(X_train_scaled,y_train)

y_pred= best_model1.predict(X_test_scaled)
r2= best_model1.score(X_test_scaled,y_test)
rmse= MSE(y_test,y_pred)**0.5

print('r2: ' + str(r2))
print('rmse: ' + str(rmse))

#r2: 0.8745999622040456
#rmse: 0.058954128844778846

joblib.dump(best_model1,'kNN_Model1.pkl')
!cp kNN_Model1.pkl drive/My\ Drive/
